In [1]:
!pip install transformers --upgrade
!pip install datasets
!pip install trl[peft] --upgrade
!pip install -U git+https://github.com/huggingface/trl
!pip install bitsandbytes loralib
!pip install wandb -U
!pip install hf_transfer




[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip
  Using cached trl-0.9.6-py3-none-any.whl (245 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.9.6.dev0
    Uninstalling trl-0.9.6.dev0:
      Successfully uninstalled trl-0.9.6.dev0

[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip
  Cloning https://github.com/huggingface/trl to /tmp/pip-req-build-zkyq7spx
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl /tmp/pip-req-build-zkyq7spx
  Resolved https://github.com/huggingface/trl to commit 82b07d6b0169bb8150f2fa4ee0a58b678d597163
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

In [2]:
%env HF_HUB_ENABLE_HF_TRANSFER=True
%env WANDB_PROJECT=LLM-Training-Course
%env WANDB_RUN_ID=ORPO
%env WANDB_NOTEBOOK_NAME={__vsc_ipynb_file__}

env: HF_HUB_ENABLE_HF_TRANSFER=True
env: WANDB_PROJECT=LLM-Training-Course
env: WANDB_RUN_ID=ORPO
env: WANDB_NOTEBOOK_NAME=/root/llm-training-course/04-dpo-orpo/03-all-in-one-training-with-orpo.ipynb


In [3]:
import wandb
wandb.login()

wandb: Currently logged in as: gal-apriori. Use `wandb login --relogin` to force relogin


True

In [4]:
import sys
sys.path.append('/root/llm-training-course/')

In [5]:
from datasets import load_dataset
train_ds, eval_ds = load_dataset("mlabonne/orpo-dpo-mix-40k", split=["train[:30%]","train[30%:35%]"])

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
train_ds

Dataset({
    features: ['source', 'chosen', 'rejected', 'prompt', 'question'],
    num_rows: 13274
})

In [7]:

columns_to_remove = [c for c in train_ds.column_names if c not in ["chosen", "rejected", "prompt"]]
orpo_train_ds = train_ds.remove_columns(columns_to_remove)
orpo_train_ds = orpo_train_ds.map()
columns_to_remove = [c for c in eval_ds.column_names if c not in ["chosen", "rejected", "prompt"]]
orpo_eval_ds = eval_ds.remove_columns(columns_to_remove)

Map: 100%|██████████| 13274/13274 [00:00<00:00, 23019.89 examples/s]


In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

In [9]:
def process_convo_for_orpo(item):
    return {
        "prompt": item["prompt"],
        "chosen": tokenizer.apply_chat_template(item["chosen"], tokenize=False),
        "rejected": tokenizer.apply_chat_template(item["rejected"], tokenize=False)
    }
orpo_train_ds = orpo_train_ds.map(process_convo_for_orpo)
orpo_eval_ds = orpo_eval_ds.map(process_convo_for_orpo)

Map: 100%|██████████| 2212/2212 [00:00<00:00, 5021.31 examples/s]


In [10]:
tokenizer.eos_token = "<|eot_id|>"
tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids(tokenizer.eos_token)
print(tokenizer)
print("---")
print("Vocab size:", tokenizer.vocab_size)
print("---")
print("Chat template:", tokenizer.chat_template)
chat_template = open('../chat_templates/llama-3-chat.jinja').read()
chat_template = chat_template.replace('    ', '').replace('\n', '')
print("Chat Template", chat_template)
tokenizer.chat_template = chat_template
print("---")


PreTrainedTokenizerFast(name_or_path='meta-llama/Meta-Llama-3-8B', vocab_size=128000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|reserved_special_token_2|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128005: AddedToken("<|reserv

In [11]:

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.31s/it]


In [12]:
from helpers import set_padding_for_tokenizer
set_padding_for_tokenizer(tokenizer)

No pad token found in vocab, using eos as  padding token


In [13]:
from helpers import stream_responses_for_sample
from transformers import GenerationConfig

generation_config =  GenerationConfig(max_new_tokens=50)
sample_conversations = [
    [{"role": "user", "content": "What is the capital of France?"}],
    [{"role": "user", "content": "Write me a javascript function that check if string is palindrome."}],
    [{"role": "user", "content": "Given x^2=36-4 what is x?"}]
]
stream_responses_for_sample(model, tokenizer, sample_conversations, generation_config=generation_config)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

What is the capital of France?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

What is the capital of France?,…

What is the capital of France?,…

What is the capital of France?,…

What is the capital of France?,…

What is the capital of France?,…

What is the capital of France?,…

What is


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Write me a javascript function that check if string is palindrome.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Write me a javascript function that check if string is palindrome.umptech.com

Write me a javascript function that check if string is palindrome.umptech.com

Write me a javascript function that check if string is palindrome.umptech.com

Write me a javascript function


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Given x^2=36-4 what is x?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

# Given x^2=36-4 what is x? suất

Given x^2=36-4 what is x? suất

## Given x^2=36-4 what is x? suất

Given x^2=36


In [14]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                     "gate_proj", "up_proj","down_proj"],
    modules_to_save=["embed_tokens", "input_layernorm", "post_attention_layernorm"],
    bias="none",
    task_type="CAUSAL_LM"
)



In [15]:
from trl import ORPOConfig, ORPOTrainer
import os
training_args = ORPOConfig(
    output_dir=os.getenv("WANDB_RUN_ID") ,
    report_to="wandb",
    num_train_epochs=1.0,
    do_train=True,
    do_eval=True,
    log_level="debug",
    gradient_checkpointing=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    per_device_eval_batch_size=4,
    lr_scheduler_type="constant",
    bf16=True,
    warmup_steps=0,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=0.2,
    max_grad_norm=.3,
    learning_rate=1e-6,
    beta=0.1,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
orpo_trainer = ORPOTrainer(
    model,
    peft_config=peft_config,
    args=training_args,
    train_dataset=orpo_train_ds,
    eval_dataset=orpo_eval_ds,
    tokenizer=tokenizer
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/orpo_trainer.py:209: UserWarning: `max_length` is not set in the ORPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/orpo_trainer.py:218: UserWarning: `max_prompt_length` is not set in the ORPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/orpo_trainer.py:247: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
Map: 100%|██████████| 2212/2212 [00:10<00:00, 202.69 examples/s]
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version

In [17]:
orpo_trainer.train()

Currently training with a batch size of: 4
***** Running training *****
  Num examples = 13,274
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 8
  Total optimization steps = 414
  Number of trainable parameters = 546,570,240
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss,Log Odds Ratio,Log Odds Chosen
83,1.491800,1.520121,503.054600,4.397000,1.099000,-0.118213,-0.130676,0.583183,0.012463,-1.306759,-1.182129,-1.108011,-1.050979,1.456356,-0.637644,0.170015
166,1.404000,1.406249,502.213300,4.405000,1.101000,-0.102805,-0.110712,0.543400,0.007907,-1.107123,-1.028050,-1.080923,-1.038597,1.340266,-0.659829,0.122055
249,1.284500,1.330799,502.121100,4.405000,1.101000,-0.092554,-0.098955,0.514467,0.006401,-0.989552,-0.925539,-1.065126,-1.030813,1.263824,-0.669750,0.109372
332,1.262100,1.304513,502.325500,4.404000,1.101000,-0.088920,-0.095140,0.509946,0.006220,-0.951398,-0.889203,-1.063881,-1.026556,1.237396,-0.671167,0.112122



***** Running Evaluation *****
  Num examples = 2212
  Batch size = 4

***** Running Evaluation *****
  Num examples = 2212
  Batch size = 4

***** Running Evaluation *****
  Num examples = 2212
  Batch size = 4

***** Running Evaluation *****
  Num examples = 2212
  Batch size = 4
Saving model checkpoint to ORPO/checkpoint-414
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B/snapshots/62bd457b6fe961a42a631306577e622c83876cb6/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm

TrainOutput(global_step=414, training_loss=1.3393338337036722, metrics={'train_runtime': 12932.0235, 'train_samples_per_second': 1.026, 'train_steps_per_second': 0.032, 'total_flos': 0.0, 'train_loss': 1.3393338337036722, 'epoch': 0.9978909310033143})